In [26]:
import pandas as pd
from bs4 import BeautifulSoup
from selenium  import webdriver
from selenium.webdriver.support.ui import Select
from webdriver_manager.chrome import ChromeDriverManager
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get('https://www.mykhel.com/kabaddi/pro-kabaddi-results/?season=1')
driver.maximize_window() 
driver.implicitly_wait(20)
elements = driver.find_elements_by_class_name("pk-fr__match-team clearfix")
soup = BeautifulSoup(driver.page_source)


Checking for win32 chromedriver:77.0.3865.40 in cache
Driver found in C:\Users\91962\.wdm\chromedriver\77.0.3865.40\win32/chromedriver.exe


In [27]:
import numpy as np
data = [b.find_all('span') for b in soup.find_all("div", {"pk-fr__match-teamdetails team-left"})]
value = []
for i in data:
    for j in i:
        value.append(j.text)
team_left = list(map(lambda s: s.strip(' '), value))
#print(team_left)
team1_df = pd.DataFrame(team_left,columns=['team1'])
#print(team_df)
team1_df.replace('', np.nan, inplace=True)
team1_df.dropna(subset=['team1'],inplace=True)
team1_df.reset_index(drop=True, inplace=True)
team1_df.head()


team1
0     Jaipur
1      Patna
2     Mumbai
3     Jaipur
4  Bengaluru

In [28]:
data = [b.find_all('span') for b in soup.find_all("div", {"pk-fr__match-teamdetails team-right"})]
value = []
for i in data:
    for j in i:
        value.append(j.text)
team_right = list(map(lambda s: s.strip(' '), value))
#print(team_left)
team2_df = pd.DataFrame(team_right,columns=['team2'])
team2_df.replace('', np.nan, inplace=True)
team2_df.dropna(inplace=True)
team2_df.reset_index(drop=True, inplace=True)
team2_df.head()

team2
0     Mumbai
1  Bengaluru
2  Bengaluru
3      Patna
4     Jaipur

In [29]:
data = soup.find_all("div",class_= "pk-fr__match-score")
#print(data)
value = []
for i in data:
    value.append(i.text)
score = list(map(lambda s: s.strip(' '), value))
#print(score)
score_df = pd.DataFrame(score,columns=['score'])
score_df.head()

score
0  35 - 24
1  29 - 22
2  27 - 23
3  38 - 18
4  30 - 29

In [30]:
data = soup.find_all("div",class_= "pk-fr__match-result")
#print(data)
value = []
for i in data:
    value.append(i.text)
winner = list(map(lambda s: s.strip(' '), value))
#print(score)
winner_df = pd.DataFrame(winner,columns=['winner'])
winner_df['winner'] = winner_df.winner.str.split(' ').str[0]
winner_df.head()

winner
0     Jaipur
1      Patna
2     Mumbai
3     Jaipur
4  Bengaluru

In [31]:
Match_Results = pd.concat([team1_df,score_df,team2_df,winner_df], axis=1, ignore_index=True)
Match_Results.columns = ['team1','score','team2','winner']
Match_Results.head()

team1    score      team2     winner
0     Jaipur  35 - 24     Mumbai     Jaipur
1      Patna  29 - 22  Bengaluru      Patna
2     Mumbai  27 - 23  Bengaluru     Mumbai
3     Jaipur  38 - 18      Patna     Jaipur
4  Bengaluru  30 - 29     Jaipur  Bengaluru

In [32]:
def team_map(x):
    return x.map({'Bengaluru':'bengaluru-bulls',
                  'Patna':'patna-pirates',
                  'Pune':'puneri-paltan',
                 'Mumbai':'u-mumba',
                 'U.P.':'up-yoddha',
                 'Gujarat':'gujarat-fortunegiants',
                 'Delhi':'dabang-delhi-kc',
                 'Haryana':'haryana-steelers',
                 'Hyderabad':'telugu-titans',
                 'Tamil Nadu':'tamil-thalaivas',
                 'Bengal':'bengal-warriors',
                 'Jaipur':'jaipur-pink-panthers',
                 'Match':'Draw'})

var = ['team1','team2','winner']
Match_Results[var] = Match_Results[var].apply(team_map)
Match_Results.head(10)

team1    score                 team2                winner
0  jaipur-pink-panthers  35 - 24               u-mumba  jaipur-pink-panthers
1         patna-pirates  29 - 22       bengaluru-bulls         patna-pirates
2               u-mumba  27 - 23       bengaluru-bulls               u-mumba
3  jaipur-pink-panthers  38 - 18         patna-pirates  jaipur-pink-panthers
4       bengaluru-bulls  30 - 29  jaipur-pink-panthers       bengaluru-bulls
5         puneri-paltan  35 - 36               u-mumba               u-mumba
6       bengaluru-bulls  27 - 26         telugu-titans       bengaluru-bulls
7       bengaluru-bulls  31 - 33         patna-pirates         patna-pirates
8       dabang-delhi-kc  45 - 22         puneri-paltan       dabang-delhi-kc
9               u-mumba  37 - 30       dabang-delhi-kc               u-mumba

In [33]:
Match_Results.to_csv("Match_Results1_raw.csv", encoding='utf-8',index=False)

In [31]:
Match_Results.loc[Match_Results.winner == Match_Results.team1,'winner']=1
Match_Results.loc[Match_Results.winner == Match_Results.team2,'winner']=2
Match_Results.loc[Match_Results.winner == 'Draw','winner']=0
Match_Results.head(10)

team1    score                  team2 winner
0       dabang-delhi-kc  39 - 39        bengaluru-bulls      0
1         patna-pirates  34 - 39       haryana-steelers      2
2  jaipur-pink-panthers  40 - 41        bengal-warriors      2
3               u-mumba  31 - 25  gujarat-fortunegiants      1
4             up-yoddha  42 - 22        tamil-thalaivas      1
5  jaipur-pink-panthers  28 - 28  gujarat-fortunegiants      0
6         puneri-paltan  42 - 38        bengaluru-bulls      1
7         telugu-titans  42 - 42          patna-pirates      0
8       bengal-warriors  48 - 36       haryana-steelers      1
9         puneri-paltan  36 - 36        tamil-thalaivas      0

In [32]:
Match_Results.to_csv("Match_Results7.csv", encoding='utf-8',index=False)

In [33]:
match = pd.read_csv("Match_Results7.csv")
match.head()